In [3]:
import pandas as pd
import csv
import dataretrieval.nwis as nwis
from datetime import date
import numpy as np
from dateutil.relativedelta import relativedelta
import plotly.express as px

class DataSchema:
    streamflow_after_consumption = 'streamflow_after_consumption'
    groundwater = 'groundwater'
    evap_rate = 'evap_rate'
    percip = 'percip'
    total_consumptive_use = 'total_consumptive_use'
    streamflow_before_consumption = 'streamflow_before_consumption'
    

class Policy:

    all_policies = []

    def __init__(self, title: str, description: str, affected: str, affect_type: str, delta: float):
        
        assert affect_type == ('proportion' or 'absolute'), f'Affect type: {affect_type}. Must be proportion or absolute.'
        # assert affected in DataSchema.__dict__, f'Affected: {affected} not in {DataSchema.__dict__}'

        if affect_type == 'proportion':
            assert delta >= 0, f'Delta: {delta}. Delta must be above 0 '

        self.title = title
        self.description = description
        self.affected = affected
        self.affect_type = affect_type
        self.delta = delta
        self.checklist_label = html.Span(children=[html.Strong(self.title), html.Br() ,self.description, html.Br()])

        Policy.all_policies.append(self)

    def __repr__(self):
        return f'{self.title}, {self.description}, {self.affected}, {self.affect_type}, {self.delta}'


    @classmethod
    def instantiate_from_csv(cls, path: str):
        with open(path,'r') as f:
            reader = csv.DictReader(f)
            policies = list(reader)
        for policy in policies:
            Policy(
                title = policy.get('Title'),
                description = policy.get('Description'),
                affected = policy.get('Affected Variable'),
                affect_type = policy.get('Affect Type'),
                delta = float(policy.get('Delta')),
            )

def LoadMonthlyStats(path: str) -> pd.DataFrame:
    '''
    Loads monthly statistics as a dataframe
    '''
    data = pd.read_csv(path,index_col=0,)
    data.rename(columns={
        'Streamflow': DataSchema.streamflow_after_consumption,
        'Groundwater': DataSchema.groundwater,
        'Evap (km3/km2)': DataSchema.evap_rate,
        'Percipitation (mm)': DataSchema.percip, 
        'Consumptive use': DataSchema.total_consumptive_use,
        'Streamflow without consumption': DataSchema.streamflow_before_consumption,
    },inplace=True)
    return data

def AdjustMonthlyStats(policies: list, monthly_stats: pd.DataFrame) -> pd.DataFrame:
    '''
    Adjusts the monthly variables by the given policies
    '''
    adjusted_monthly = monthly_stats.copy(deep=True)  

    for policy in policies:
        if policy.affect_type == 'proportion':
            adjusted_monthly[policy.affected] *= policy.delta
    for policy in policies:
        if policy.affect_type == 'absolute':
            adjusted_monthly[policy.affected] += policy.delta
    
    adjusted_monthly[DataSchema.total_consumptive_use].clip(0,inplace=True)

    adjusted_monthly[DataSchema.streamflow_after_consumption] = (
        adjusted_monthly[DataSchema.streamflow_before_consumption] - adjusted_monthly[DataSchema.total_consumptive_use])
    
    return adjusted_monthly

def GetUSGSSiteData(site_num: str, start_date: str, end_date: str, service='dv') -> pd.DataFrame:
    '''
    Uses dataretrieval.nwis to load the record of the given site
    '''
    df = nwis.get_record(sites=site_num, service=service ,start=start_date, end=end_date)
    df.reset_index(inplace=True)
    return df

def CreateLakeData(path: str, bath_df: pd.DataFrame) -> pd.DataFrame:
    '''
    This function loads the historic data from the saved cvs and then updates with new data if required
    '''
    today = date.today().strftime("%Y-%m-%d")

    saved_data = pd.read_csv(path, index_col=0)
    last_saved_day = saved_data.at[len(saved_data)-1, 'datetime']

    if saved_data.at[len(saved_data)-1, 'datetime'].split('-')[1] == today.split('-')[1]:
        return saved_data

    today = date.today().strftime("%Y-%m-%d")
    FOOT_TO_METER = 0.3048
    
    new_data = GetUSGSSiteData('10010024',last_saved_day,today)

    new_data['YYYY-MM'] = new_data['datetime'].dt.year.astype(str) + '-' + new_data['datetime'].dt.month.astype(str) #the better way
    new_data = new_data.groupby(new_data['YYYY-MM']).mean()
    new_data = new_data.iloc[1:] 

    new_data.rename(columns={'62614_Mean':'Elevation'},inplace=True)
    new_data['Elevation'] *= FOOT_TO_METER
    new_data.reset_index(inplace=True)
    new_data['datetime'] = pd.to_datetime(new_data['YYYY-MM'])

    for index, row in new_data.iterrows():
        round_e = round(new_data.at[index,'Elevation'], 2)
        new_data.at[index,'Surface Area'] = bath_df.at[round_e,'Surface Area']
        new_data.at[index,'Volume'] = bath_df.at[round_e,'Volume']

    combined = pd.concat([saved_data,new_data],ignore_index=True)
    combined.reset_index(inplace=True,drop=True)

    return combined

def GSLPredictor(years_forward: int, monthly_stats: pd.DataFrame,
                    bath_df: pd.DataFrame, lake_df: pd.DataFrame, start_date='',
                    weather=False):
    
    MAX_VOLUME = 39.896064
    
    if len(start_date) == 0:
        yyyy_mm_date = date.today().strftime("%Y-%m")  
    else:
        yyyy_mm_date = start_date
        
    cur_date = pd.to_datetime(yyyy_mm_date)

    months_forward = 12 * years_forward
    
    elevation = round(lake_df.set_index('YYYY-MM').at[yyyy_mm_date,'Elevation'], 2)
    predictions = pd.DataFrame(columns=['datetime','Elevation Prediction','YYYY-MM'])
    
    weather_factor = 0
    lr_weather_counter = 0
    
    for i in range(months_forward):
        surface_area = bath_df.at[elevation,'Surface Area']
        volume = bath_df.at[elevation,'Volume']
        month_index = (i%12)
        
        lost_evap = surface_area * monthly_stats.at[month_index,DataSchema.evap_rate]
        gained_rain = surface_area * (monthly_stats.at[month_index,DataSchema.percip]/1000000)
        gained_stream = monthly_stats.at[month_index,DataSchema.streamflow_after_consumption]
        
        if weather:
            if lr_weather_counter == 0:
                lr_weather_counter = np.random.randint(60,120)
                lr_weather_factor = np.random.normal(scale=0.15)
                
            if month_index == 0:
                weather_factor = np.random.normal(scale=0.35)
            
            lr_weather_counter -= 1
            
            gained_rain *= 1 + (weather_factor + lr_weather_factor)
            gained_stream *= 1 + (weather_factor +  lr_weather_factor)
        
        gained_ground = 0.00775
        net = gained_rain + gained_stream + gained_ground - lost_evap
        
        volume += net
        
        if volume >= MAX_VOLUME:
            elevation = bath_df.index[(bath_df['Volume']-MAX_VOLUME).abs().argsort()][:1][0]
        else:
            elevation = bath_df.index[(bath_df['Volume']-volume).abs().argsort()][:1][0]
            
        cur_date += relativedelta(months=1)
        
        predictions.loc[len(predictions.index)] = [cur_date,elevation,cur_date.strftime("%Y-%m")]
        
    return predictions

def CreateLineGraph(prediction: pd.DataFrame, lr_average_elevaton: float, df_lake: pd.DataFrame, rolling=60) -> px.scatter:

    MEAN_ELEVATION_WITHOUT_HUMANS = 1282.3

    combined = pd.concat([df_lake,prediction],ignore_index=True)

    temp = pd.concat([combined.iloc[len(df_lake)-rolling:len(df_lake)]['Elevation'],combined.iloc[len(df_lake):]['Elevation Prediction']])
    combined['Elevation Prediction'] = temp

    colors = ['blue','red']

    combined['datetime'] = pd.to_datetime(combined['datetime'])

    fig = px.scatter(combined, y=['Elevation','Elevation Prediction'],
                        x='datetime', trendline='rolling',
                        trendline_options=dict(window=rolling),
                        color_discrete_sequence=colors,
                        labels = {
                            'value':'Lake Elevation (m)',
                            'datetime':'Year'
                        },
                    )

    start_date = "1870-01-01"
    end_date = combined.at[len(combined)-1, "datetime"]
    fig.update_xaxes(type='date', range=[start_date, end_date])

    #only show trendlines
    fig.data = [t for t in fig.data if t.mode == 'lines']

    #assign label positions
    if (0 < (MEAN_ELEVATION_WITHOUT_HUMANS - lr_average_elevaton) < 0.4) or (0 < (df_lake['Elevation'].mean() - lr_average_elevaton) < 0.4):
        lr_pos = 'bottom left'
        human_pos = 'top left'
    elif 0 < (lr_average_elevaton - df_lake['Elevation'].mean()) < 0.4:
        lr_pos = 'top left'
        human_pos = 'bottom left'
    else:
        lr_pos = 'top left'
        human_pos = 'top left'

    fig.add_hline(y=MEAN_ELEVATION_WITHOUT_HUMANS, line_dash='dot',
                    annotation_text = f'Average Natural Level, {MEAN_ELEVATION_WITHOUT_HUMANS}m',
                    annotation_position = 'top left',
                    annotation_font_size = 10,
                    annotation_font_color = 'black',
                    )
    avg_elevation = round(df_lake['Elevation'].mean(),2)
    fig.add_hline(y=avg_elevation, line_dash='dot',
                annotation_text = f'Average Since 1847, {avg_elevation}m',
                annotation_position = human_pos,
                annotation_font_size = 10,
                annotation_font_color = colors[0],
                line_color = colors[0],
                )

    fig.add_hline(y=lr_average_elevaton, 
                    line_dash='dot',
                    annotation_text = f'Long-term Policy Average, {lr_average_elevaton}m',
                    annotation_position = lr_pos,
                    annotation_font_size = 10,
                    annotation_font_color = colors[1],
                    line_color = colors[1],
            )

    return fig

def WrittenEffects(lr_average_elevation: float, df_lake: pd.DataFrame, bath_df: pd.DataFrame) -> html.Div:
    NATURAL_ELEVATION_MEAN = 1282.30
    NATURAL_SA_MEAN = bath_df.at[NATURAL_ELEVATION_MEAN, 'Surface Area']
    NATURAL_VOLUME_MEAN = bath_df[NATURAL_ELEVATION_MEAN, 'Volume']
    HUMAN_ELEVATION_MEAN = round(df_lake['Elevation'].mean(), 2)

    delta_elevation_percent = 100 * ((lr_average_elevation - NATURAL_ELEVATION_MEAN) / NATURAL_ELEVATION_MEAN)
    delta_sa_percent = 100 * ((bath_df.at[lr_average_elevation, 'Surface Area'] - NATURAL_SA_MEAN) / NATURAL_SA_MEAN)
    delta_volume_percent = 100 * ((bath_df.at[lr_average_elevation, 'Volume'] - NATURAL_VOLUME_MEAN) / NATURAL_VOLUME_MEAN)


    if delta_elevation_percent >= 0:
        color = 'green'
        elevation_descriptor = 'higher'
        volume_descriptor = 'more'
    else:
        color = 'red'
        descriptor = 'lower'
        volume_descriptor = 'less'
    
    return html.Div(
        id = 'written-effects',
        children=[
            html.H3('Based on the selected policy choices, in the long term, the lake will be:'),
            html.Ul([
                html.Li(f'fds',style={'color':color})
            ])
        ]
    )

def BuyBackEffect(millions_spent: int) -> float:
    '''
    This function takes the amount of money spent on water rights buy backs and calculates the total amount of water saving it would create, in km3/yr
    '''
    pass

def RetrieveImage(lr_average_elevation: float) -> html.Img:
    closest_half_meter = round(lr_average_elevation * 2) / 2
    image_path = f'/assets/gsl_{closest_half_meter}.png'
    return html.Img(src=image_path)


Policy.instantiate_from_csv('data/policies.csv')
monthly_stats = LoadMonthlyStats('data/monthly_stats.csv')
bath = pd.read_csv('data/bath_df.csv', index_col=0)
lake = CreateLakeData('data/lake_df.csv', bath)


NameError: name 'html' is not defined

In [20]:
lake = pd.read_csv('data/lake_df.csv')
lake['datetime'] = pd.to_datetime(lake['datetime'])
lake_yearly = lake.groupby(lake['datetime'].dt.year).mean().reset_index()
px.scatter(lake_yearly, x='datetime', y='Elevation', trendline='ols')
# lake_yearly

In [12]:
use_vs_elevation = pd.DataFrame(columns = ['Consumptive Use','Longrun Elevation'])
for i in range(21):
    adjusted_monthly = monthly_stats.copy(deep=True)
    adjusted_monthly[DataSchema.streamflow_after_consumption] = adjusted_monthly[DataSchema.streamflow_before_consumption] * (i/20)
    year_consumptive_use = (adjusted_monthly[DataSchema.streamflow_before_consumption] - adjusted_monthly[DataSchema.streamflow_after_consumption]).sum()
    prediction = GSLPredictor(50, adjusted_monthly, bath, lake)
    lr_average = prediction.tail(12)['Elevation Prediction'].mean()
    use_vs_elevation.loc[len(use_vs_elevation)] = [year_consumptive_use, lr_average]
use_vs_elevation

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


,Consumptive Use,Longrun Elevation
0,4.358223,1270.575000
1,4.140312,1270.958333
2,3.922401,1271.442500
3,3.704490,1272.146667
4,3.486578,1273.050833
5,3.268667,1274.010833
6,3.050756,1274.974167
7,2.832845,1276.019167
8,2.614934,1277.099167
9,2.397023,1278.032500


In [75]:
fig = px.line(
    use_vs_elevation,
    title = 'Effect of consumptive use on long run lake elevation',
    x = 'Consumptive Use', 
    y = 'Longrun Elevation',
    labels = {
        'Elevation': 'Elevation (m)',
        'Consumptive Use': 'Consumptive Use (km3/yr)'
    }
)

CURRENT_CONSUMPTIVE_USE = 1.77
fig.add_vline(
    x = CURRENT_CONSUMPTIVE_USE,
    annotation_text = f"Current consumptive use: {CURRENT_CONSUMPTIVE_USE}"
)


fig.show()

In [2]:
lake

NameError: name 'lake' is not defined

In [81]:
import plotly.graph_objects as go

labels = [
    'Streamflow',
    'Bear River',
    'Weber River',
    'Jordan River',
    'Groundwater',
    'Direct Percipitation',
    'Total Water In',
    'Mineral Extraction',
    'Evaporation',
    'Lake Water Lost',
    'Total Water Out',
    'Other Streams'
]

#Data is average from 1963 to 2022. Mineral extraction data is from 1994 to 2014. Other stream data is assumed to be linearly dependent upon the other three streams

units = "km^3/yr"

fig = go.Figure(
    data=go.Sankey(
        arrangement = "snap",
        valuesuffix = units,
        node = dict(
            label = labels,
            x = [0.25, 0, 0, 0, 0.25, 0.25, 0.5, 1, 1, 0.5, 0.75, 0],
            y = [0.5] * 10,
            # y = [1, 1, 0.9, 0.8, 0.8, 0.9, 1, 0.9, 1, 0.9, 1, 0.7],
            hovertemplate = "%{label}"
        ),
        link = dict(
            source = [1, 2, 3, 0, 4, 5, 10, 10, 6, 9, 11],
            target = [0, 0, 0, 6, 6, 6, 7, 8, 10, 10, 0],
            value = [1.59, 0.377, 0.52, 2.63, 0.09, 1.47, 0.19, 4.25, 4.19, 0.25, 0.14],
            
        ),
))
fig.show()

In [77]:
consumptive_use_sunburst_df = pd.DataFrame(columns=['general_category','specific_category'])


In [119]:
yearly_df = pd.read_csv('data/yearly_pop_and_consumption.csv')
yearly_df['Population'] *= 1000
yearly_df['datetime'] = pd.to_datetime(yearly_df['Year'].astype('str'))
yearly_df

,Year,Agriculture,Muncipal,Reservoir,Mineral,Wetland,Imports,Net Depletion,Population,datetime
0,1850,0.00,0.0,0.0,0.0,0.0,0.0,0.00,1.188000e+04,1850-01-01
1,1851,0.03,0.0,0.0,0.0,0.0,0.0,0.03,1.342259e+04,1851-01-01
2,1852,0.06,0.0,0.0,0.0,0.0,0.0,0.06,1.516547e+04,1852-01-01
3,1853,0.09,0.0,0.0,0.0,0.0,0.0,0.09,1.713467e+04,1853-01-01
4,1854,0.12,0.0,0.0,0.0,0.0,0.0,0.12,1.935956e+04,1854-01-01
...,...,...,...,...,...,...,...,...,...,...
167,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.103540e+06,2017-01-01
168,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.155153e+06,2018-01-01
169,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.203383e+06,2019-01-01
170,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.281684e+06,2020-01-01


In [126]:
yearly_df = pd.read_csv('data/yearly_pop_and_consumption.csv')
yearly_df['Population'] *= 1000
yearly_df['datetime'] = pd.to_datetime(yearly_df['Year'].astype('str'))

for column in yearly_df:
    if column == 'Year' or column == 'datetime':
        pass
    else:
        label = f'{column}_scaled'
        yearly_df[label] = (yearly_df[column] / yearly_df[column].max()) * 100

yearly_df

,Year,Agriculture,Muncipal,Reservoir,Mineral,Wetland,Imports,Net Depletion,Population,datetime,Agriculture_scaled,Muncipal_scaled,Reservoir_scaled,Mineral_scaled,Wetland_scaled,Imports_scaled,Net Depletion_scaled,Population_scaled
0,1850,0.00,0.0,0.0,0.0,0.0,0.0,0.00,1.188000e+04,1850-01-01,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.355904
1,1851,0.03,0.0,0.0,0.0,0.0,0.0,0.03,1.342259e+04,1851-01-01,1.796407,0.0,0.0,0.0,0.0,0.0,1.310044,0.402118
2,1852,0.06,0.0,0.0,0.0,0.0,0.0,0.06,1.516547e+04,1852-01-01,3.592814,0.0,0.0,0.0,0.0,0.0,2.620087,0.454332
3,1853,0.09,0.0,0.0,0.0,0.0,0.0,0.09,1.713467e+04,1853-01-01,5.389222,0.0,0.0,0.0,0.0,0.0,3.930131,0.513325
4,1854,0.12,0.0,0.0,0.0,0.0,0.0,0.12,1.935956e+04,1854-01-01,7.185629,0.0,0.0,0.0,0.0,0.0,5.240175,0.579979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.103540e+06,2017-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92.976730
168,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.155153e+06,2018-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.522967
169,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.203383e+06,2019-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.967855
170,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.281684e+06,2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.313618


In [128]:
fig = px.scatter(
    yearly_df, 
    x='datetime', 
    y=['Net Depletion_scaled','Population_scaled','Agriculture_scaled','Municipal_scaled'],
    trendline='rolling', 
    trendline_options=dict(window=5),
    labels = {
        'value':'Percent of maximum',
        'datetime':'Year'
    },
)
fig.data = [t for t in fig.data if t.mode == 'lines']
start_date = "1960-01-01"
end_date = "2020-01-01"
fig.update_xaxes(type='date', range=[start_date, end_date])
fig.show()

ValueError: All arguments should have the same length. The length of argument `y` is 4, whereas the length of  previously-processed arguments ['datetime'] is 172

In [109]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go


fig = make_subplots(specs=[[{'secondary_y':True}]])

fig.add_trace(
    go.Scatter(x=yearly_df['datetime'],y=yearly_df['Agriculture'], name='Agriculture'),
    secondary_y = False,
)

fig.add_trace(
    go.Scatter(x=yearly_df['datetime'],y=yearly_df['Population'], name='Population'),
    secondary_y = True,
)

start_date = "1960-01-01"
end_date = "2020-01-01"
fig.update_xaxes(type='date', range=[start_date, end_date])

fig.update_layout(
    hover
)

fig.show()